<a href="https://colab.research.google.com/github/Edu-Moreira/Estudos-de-Python/blob/main/JQ_Backtest_Autocorrela%C3%A7%C3%A3o_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance
!pip install vectorbt
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 526 kB 7.3 MB/s 
     |████████████████████████████████| 288 kB 41.2 MB/s 
     |████████████████████████████████| 1.6 MB 36.9 MB/s 
     |████████████████████████████████| 749 kB 22.8 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70 kB 4.9 MB/s 


In [3]:
import numpy as np 
import yfinance as yf
import pandas as pd 
import vectorbt as vbt
import mplfinance as mpf
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

#Parametros
tickers = ['ITUB4.SA','PETR4.SA','MGLU3.SA','BTC-USD']
janela = 10

lista_dados = []

for i  in tickers:
    ticker_ohlc = yf.download(i,start='2012-07-01', end = '2022-06-30')
    ticker_ohlc['autocorr'] = ticker_ohlc['Close'].rolling(janela).apply(lambda x: x.autocorr(), raw = False)
    lista_dados.append(ticker_ohlc)
    
lista_dados[2]


#criando benchmark

buy_hold = []

for i in range(0, len(tickers)):
    entradas = lista_dados[i]['Close'].iloc[0]
    saidas = lista_dados[i]['Close'].iloc[-1]
    retornos = (saidas/entradas) -1
    buy_hold.append(retornos)
    
bh = pd.DataFrame(buy_hold, columns = ['Buy&Hold'])
bh

#Gatilhos de entrada
lista_entradas_autocorr = []
lista_saidas_autocorr = []


for i in range(0,len(tickers)):
    entradas = (lista_dados[i]['autocorr']> 0.8).vbt.signals.AND(lista_dados[i]['Close']>lista_dados[i]['Close'].shift(janela))
    saidas = (lista_dados[i]['autocorr']> 0.8).vbt.signals.AND(lista_dados[i]['Close']<lista_dados[i]['Close'].shift(janela))
        
    lista_entradas_autocorr.append(entradas)
    lista_saidas_autocorr.append(saidas)


#Criando BACKTEST

lista_resultados_autocorr = []

for i in range (0, len(tickers)):
    estrategia_autocorr = vbt.Portfolio.from_signals(lista_dados[i].Close, lista_entradas_autocorr[i], lista_saidas_autocorr[i],direction='longonly', price = lista_dados[i].Open.vbt.bshift(1))
    lista_resultados_autocorr.append(estrategia_autocorr)
    
#Criar um log para relatório
lista_positions_autocorr = []

for i in range (0, len(tickers)):
    positions = vbt.Portfolio.from_signals(lista_dados[i].Close, lista_entradas_autocorr[i], lista_saidas_autocorr[i],direction='longonly', price = lista_dados[i].Open.vbt.bshift(1)).positions
    lista_positions_autocorr.append(positions)
    
    
#Lista Retornos
lista_retornos_autocorr = []

for i in range(0,len(tickers)):
    total_return_autocorr = (tickers[i],lista_resultados_autocorr[i].total_return())
    lista_retornos_autocorr.append(total_return_autocorr)
    
overview = pd.DataFrame(lista_retornos_autocorr, columns = ['Ativos', 'Retorno'])
overview = pd.merge(overview, bh, how = 'inner', left_index= True, right_index = True)
overview.index = overview.Ativos
overview.drop(['Ativos'], axis = 1 , inplace= True)
overview
    
fig = ff.create_table(overview, height_constant= 60 , index = True)
fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'height':300 , 'width' : 500})
fig.show()


#Analise de Consistencia 
lista_resultados_autocorr[0].plot(subplots=['value'], width = 600, height =400).show()



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:

#tabela com as osrdens executados 

posicao_1 = lista_positions_autocorr[0].records_readable
posicao_2 = lista_positions_autocorr[1].records_readable
posicao_3 = lista_positions_autocorr[2].records_readable
posicao_4 = lista_positions_autocorr[3].records_readable

posicao_1.groupby(['Direction']).sum()['PnL']


fig = make_subplots(rows =2,
                    cols =2,
                    row_heights=[5,5],
                    column_width = [3,3],
                    subplot_titles = (tickers[0],tickers[1],tickers[2],tickers[3]),
                    shared_xaxes = False)
fig.add_trace(go.Histogram(x=posicao_1['Return'],nbinsx = 30),row=1,col=1)
fig.add_trace(go.Histogram(x=posicao_2['Return'],nbinsx = 30),row=1,col=2)
fig.add_trace(go.Histogram(x=posicao_3['Return'],nbinsx = 30),row=2,col=1)
fig.add_trace(go.Histogram(x=posicao_4['Return'],nbinsx = 30),row=2,col=2)
fig.update_layout(title_text = '<b>Avaliação Retornos<b>',
                  template = 'simple_white',
                  showlegend = False,
                  height= 500,
                  width=900)
fig.show()

In [6]:

##Analise do PnL

y = [posicao_1['PnL'].sum(),posicao_2['PnL'].sum(),posicao_3['PnL'].sum(),posicao_4['PnL'].sum()]

fig = go.Figure()
fig.add_trace(go.Bar(x=tickers, y=y,text= y, textposition = 'outside', textfont_color = 'black'))
fig.update_layout(title_text = '<b>Pnl Analysis', template = 'simple_white',height = 500, width = 900)
fig.update_traces(marker_color ='rgb(158,202,225)',marker_line_color = 'rgb(8,48,107)',marker_line_width = 1.5, opacity = 0.6)

fig.show()
